In [ ]:
import json
import paho.mqtt.client as mqtt
import time
import threading
import base64
import requests
import queue

In [ ]:
class MQTTClient:
    def __init__(self, target_ip, default_port, sub_topic, pub_topic):
        """
        初始化
        :param target_ip: mqtt服务器IP
        :param default_port: mqtt端口号
        :param sub_topic: sub的topic
        :param pub_topic: pub的topic
        """
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.client.connect(target_ip, default_port, 5)
        self.pub_topic = pub_topic
        self.client.subscribe(sub_topic, qos=0)
        self.client.loop_start()
        self.message = ""

    def on_connect(self, client, userdata, flags, rc):
        """
        连接功能
        :param client:
        :param userdata:
        :param flags:
        :param rc:
        :return:
        """
        pass

    def on_message(self, client, userdata, msg):
        """
        接收数据
        :param client:
        :param userdata:
        :param msg: 接收到的信息
        :return: none
        """
        self.message = json.loads(msg.payload.decode())

    def send_mqtt(self, data):
        """
        发送mqtt
        :param data: 发送的数据
        :return: none
        """
        self.client.publish(self.pub_topic, payload="{}".format(data), qos=0)

    def control_device(self, key, value):
        print("-------------")
        self.send_message(json.dumps({key : value}))


In [ ]:
class FS_IIOTA:
    def __init__(self, mqtt_broker_addr, cv_edge_addr, detection_labels):
        self.mqtt_client = MQTTClient(mqtt_broker_addr, 1883, "sim_device_01/device_feedback", "sim_device_01/device_control")
        self.cv_edge_addr = cv_edge_addr
        self.detection_labels = detection_labels
        self.q_enable_rod_pull = queue.Queue()
        self.t_enable_rod_pull = threading.Thread(target = self.enable_rod_pull)
        self.t_enable_rod_pull.start()
        self.mqtt_client.send_mqtt(json.dumps({"conveyor":"run"}))
        self.mqtt_client.send_mqtt(json.dumps({'rod_control': "all_pull"}))
        time.sleep(3)
        self.mqtt_client.send_mqtt(json.dumps({'rod_control': "first_push"}))
        self.q_enable_rod_pull.put("first_pull")
        time.sleep(0.0001)
        self.detection_num = 9
        self.str_positionObject = 'Before1st'

    def handle_mqtt_rcv(self):
        while True:
            if self.mqtt_client.message != '':
                if "first_switch" in self.mqtt_client.message:
                    if self.mqtt_client.message['first_switch'] == True:
                        self.str_positionObject = 'At1st'
                    elif self.mqtt_client.message['first_switch'] == False and self.str_positionObject == 'At1st':
                        self.str_positionObject = 'Between1stAnd2nd'
                        if self.detection_num == 0:
                            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "second_push"}))
                            self.q_enable_rod_pull.put("second_pull")
                if "second_switch" in self.mqtt_client.message:
                    if self.mqtt_client.message['second_switch'] == True:
                        self.str_positionObject = 'At2nd'
                    elif self.mqtt_client.message['second_switch'] == False and self.str_positionObject == 'At2nd':
                        self.str_positionObject = 'Between2ndAnd3rd'
                        if self.detection_num == 1:
                            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "third_push"}))
                            self.q_enable_rod_pull.put("third_pull")
                if "third_switch" in self.mqtt_client.message:
                    if self.mqtt_client.message['third_switch'] == True:
                        self.str_positionObject = 'At3rd'
                    elif self.mqtt_client.message['third_switch'] == False and self.str_positionObject == 'At3rd':
                        self.str_positionObject = 'Beyond3rd'
                        if self.detection_num == 2:
                            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "fourth_push"}))
                            self.q_enable_rod_pull.put("fourth_pull")
                        else:
                            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "first_push"}))
                if "image" in self.mqtt_client.message:
                    imgData = base64.b64decode(self.mqtt_client.message["image"])
                    try:
                        result = json.loads(requests.post(self.cv_edge_addr, data=imgData).text)
                        print("识别结果：", result)
                        
                        try:
                            result_index = 0
                            if len(result["results"]) > 1:
                                result_index = max(range(len(result["results"])), key=lambda index: result["results"][index]['confidence'])
                            self.detection_num = self.detection_labels.index(result["results"][result_index]["label"])
                        except:
                            pass
                    except:
                        print("无法post。")
                self.mqtt_client.message = ''

    def enable_rod_pull(self):
        while True:
            value = self.q_enable_rod_pull.get()
            print("value",value)
            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "first_push"}))
            time.sleep(3)
            self.mqtt_client.send_mqtt(json.dumps({'rod_control': value}))
            self.mqtt_client.send_mqtt(json.dumps({'rod_control': "first_pull"}))

In [ ]:
cv_edge_addr = 'http://127.0.0.1:8088/aisim_tf_pre'
mqtt_broker_addr = "127.0.0.1"  # Windows的broker的IP地址
fs = FS_IIOTA(mqtt_broker_addr, cv_edge_addr, ["ripe", "half_ripe", "raw"])
fs.handle_mqtt_rcv()